In [84]:
import pandas as pd
import json
from mplsoccer import Pitch
from datetime import datetime

DATA_FOLDER = "data"

# Get the matches

In [89]:
# Get all matches
matches_df = pd.read_parquet(f"{DATA_FOLDER}/matches.parquet")

# Create new column with match name (team names)
matches_df["match"] = (
    matches_df["home_team"].apply(lambda x: x["short_name"])
    + " vs "
    + matches_df["away_team"].apply(lambda x: x["short_name"])
)

# Create data column with match date
matches_df["date"] = pd.to_datetime(matches_df["date_time"]).dt.date

# Sort by date
matches_df = matches_df.sort_values('date_time', ascending=True)

matches_df.tail()

# Get games from 2024-2025 season
start_date_season_24_25 = datetime.strptime("2024-08-10", "%Y-%m-%d").date()
season_24_25_matches = matches_df[matches_df["date"] > start_date_season_24_25]

# Get games from 2023-2024 season
season_23_24_matches = matches_df[matches_df["date"] < start_date_season_24_25]

,id,date_time,home_team,away_team,status,competition_id,season_id,competition_edition_id,match,date
56,1528510,2024-05-11T16:30:00Z,"{'id': 283, 'short_name': 'Granada CF'}","{'id': 262, 'short_name': 'Real Madrid'}",closed,4,28,546,Granada CF vs Real Madrid,2024-05-11
55,1534700,2024-05-14T19:30:00Z,"{'id': 262, 'short_name': 'Real Madrid'}","{'id': 313, 'short_name': 'Alavés'}",closed,4,28,546,Real Madrid vs Alavés,2024-05-14
54,1543050,2024-05-19T17:00:00Z,"{'id': 260, 'short_name': 'Villarreal CF'}","{'id': 262, 'short_name': 'Real Madrid'}",closed,4,28,546,Villarreal CF vs Real Madrid,2024-05-19
53,1552423,2024-05-25T19:00:00Z,"{'id': 262, 'short_name': 'Real Madrid'}","{'id': 269, 'short_name': 'Betis Balompié'}",closed,4,28,546,Real Madrid vs Betis Balompié,2024-05-25
52,1562415,2024-06-01T19:00:00Z,"{'id': 103, 'short_name': 'Dortmund'}","{'id': 262, 'short_name': 'Real Madrid'}",closed,10,28,569,Dortmund vs Real Madrid,2024-06-01


# Get possession chains for a certain match
For example: the last game in the dataset:
- id: 2018396
- match: Real Madrid vs Real Sociedad
- date: 2025-05-24

For a more detailed explanation: check the [Dynamic Events CSV Specifications file](./20250314%20-%20Dynamic%20Events%20CSV%20Specifications.pdf)

In [94]:
def get_possession_chains(match_id):
    dynamic_data = pd.read_parquet(f"{DATA_FOLDER}/dynamic/{match_id}.parquet")
    
    # Only keep player_possession events
    player_possession_df = dynamic_data[dynamic_data["event_type"] == "player_possession"]

    # Relevant columns for possession chains
    pos_chain_cols = [
        "index", "match_id", "team_shortname",
        "phase_index",
        "channel_start", "channel_end", 
        "third_start", "third_end", 
        "penalty_area_start", "penalty_area_end",
        "game_interruption_after", "pass_direction_received", "pass_outcome",
        "lead_to_shot", "lead_to_goal", "possession_danger",
        "pass_ahead", "dangerous",
        "first_line_break", "second_last_line_break", "last_line_break", "furthest_line_break",
        "n_opponents_bypassed"
    ]

    return player_possession_df[pos_chain_cols]

pos_chain_df = get_possession_chains(2018396)
pos_chain_df.head()

,index,match_id,team_shortname,phase_index,channel_start,channel_end,third_start,third_end,penalty_area_start,penalty_area_end,...,lead_to_shot,lead_to_goal,possession_danger,pass_ahead,dangerous,first_line_break,second_last_line_break,last_line_break,furthest_line_break,n_opponents_bypassed
0,0,2018396,Real Madrid,0,center,center,middle_third,middle_third,False,False,...,False,False,None,False,None,False,False,False,None,-3.0
3,3,2018396,Real Madrid,0,center,center,middle_third,middle_third,False,False,...,False,False,None,False,None,False,False,False,None,-1.0
6,6,2018396,Real Madrid,0,half_space_left,half_space_left,middle_third,middle_third,False,False,...,False,False,None,True,None,False,False,False,None,0.0
14,14,2018396,Real Madrid,0,half_space_right,half_space_right,middle_third,middle_third,False,False,...,False,False,None,True,None,True,False,False,first,2.0
19,19,2018396,Real Madrid,0,half_space_right,half_space_right,middle_third,middle_third,False,False,...,False,False,None,False,None,False,False,False,None,-2.0


# Score parts of possession chain

In [97]:
# Possible scoring metrics:

# third_end == "attacking_third"
# penalty_area_end = True
# game_interruption_after == "corner_for", "free_kick_for", "goal_for", "penalty_for", "throw_in_for"
# pass_direction_received == "forward"
# pass_outcome == "successful"
# lead_to_shot == True
# lead_to_goal == True
# possession_danger == True
# pass_ahead == True
# dangerous == True
# first_line_break == True
# second_last_line_break == True
# last_line_break == True
# furthest_line_break == True
# n_opponents_bypassed == (int)

def score_possession_chain(chain_df):
    # Init channel scores
    channel_scores = {}

    # Loop through events of current chain
    for index, row in chain_df.iterrows():
        start_channel = row["channel_start"]
        end_channel = row["channel_end"]

        # Add 1 point to start channel
        if start_channel in channel_scores:
            channel_scores[start_channel] += 1
        else:
            channel_scores[start_channel] = 1

        # Add 1 point to end channel
        if end_channel in channel_scores:
            channel_scores[end_channel] += 1
        else:
            channel_scores[end_channel] = 1

        # Calculate total points
        total_points = sum(channel_scores.values())

        # Calculate percentages
        channel_percentages = {channel: (points / total_points) * 100 for channel, points in channel_scores.items()}

        # Calculate wing and central scores/percentages
        wing_channels = ["wide_right", "wide_left"]
        central_channels = ["center"]

        wing_score = sum(channel_scores.get(channel, 0) for channel in wing_channels)
        central_score = sum(channel_scores.get(channel, 0) for channel in central_channels)

        # Adjust scores with half space channels
        half_space_channels = ["half_space_right", "half_space_left"]
        half_space_score = sum(channel_scores.get(channel, 0) for channel in half_space_channels)
        wing_score += half_space_score / 2
        central_score += half_space_score / 2

        wing_percentage = wing_score / total_points
        central_percentage = central_score / total_points

        # Classification logic
        if wing_percentage > 0.5:
            classification = "wing"
        elif central_percentage > 0.5:
            classification = "central"
        else:
            classification = "mixed"

    return {
        'channel_scores': channel_scores,
        'channel_percentages': channel_percentages,
        'wing_score': wing_score,
        'central_score': central_score,
        'total_points': total_points,
        'wing_percentage': wing_percentage,
        'central_percentage': central_percentage,
        'classification': classification
    }

pos_chain_test = pos_chain_df[pos_chain_df["phase_index"] == 0]
score_possession_chain(pos_chain_test)

{'channel_scores': {'center': 11,
  'half_space_left': 2,
  'half_space_right': 13,
  'wide_right': 4,
  'wide_left': 2},
 'channel_percentages': {'center': 34.375,
  'half_space_left': 6.25,
  'half_space_right': 40.625,
  'wide_right': 12.5,
  'wide_left': 6.25},
 'wing_score': 13.5,
 'central_score': 18.5,
 'total_points': 32,
 'wing_percentage': 0.421875,
 'central_percentage': 0.578125,
 'classification': 'central'}

# Analyze all possession chains in a game

In [99]:
def analyze_match_chains(data):
    madrid_chains = data[data["team_shortname"] == "Real Madrid"]["phase_index"].unique()
    chain_results = []

    for chain_id in madrid_chains:
        chain_df = data[data["phase_index"] == chain_id]
        chain_result = score_possession_chain(chain_df)

        # Chains have to be at least 3 events long
        chain_length = chain_df.shape[0]
        if chain_length < 3:
            continue

        # Add chain data to result
        chain_results.append({
            "match_id": chain_df["match_id"].iloc[0],
            "chain_id": chain_id,
            "chain_length": chain_length,
            "classification": chain_result["classification"],
            "wing_percentage": chain_result["wing_percentage"],
            "central_percentage": chain_result["central_percentage"]
        })

    return pd.DataFrame(chain_results)


chains_test_game = analyze_match_chains(pos_chain_df)
chains_test_game

,match_id,chain_id,chain_length,classification,wing_percentage,central_percentage
0,2018396,0,16,central,0.421875,0.578125
1,2018396,2,4,wing,0.562500,0.437500
2,2018396,9,4,wing,0.625000,0.375000
3,2018396,12,8,central,0.250000,0.750000
4,2018396,15,9,wing,0.611111,0.388889
...,...,...,...,...,...,...
86,2018396,298,7,wing,0.750000,0.250000
87,2018396,301,3,mixed,0.500000,0.500000
88,2018396,303,6,wing,0.708333,0.291667
89,2018396,307,3,central,0.333333,0.666667


In [100]:
mean_wing = chains_test_game['wing_percentage'].mean()
mean_central = chains_test_game['central_percentage'].mean()
print(f"\nAverage wing play: {mean_wing:.2f}%")
print(f"Average central play: {mean_central:.2f}%")


Average wing play: 0.51%
Average central play: 0.49%


In [101]:
chain_types = chains_test_game['classification'].value_counts(normalize=True)
print("Proportion of each chain type:")
print(chain_types)

Proportion of each chain type:
classification
wing       0.494505
central    0.428571
mixed      0.076923
Name: proportion, dtype: float64


# Analyzing whole seasons

In [ ]:
# all games = matches_df
